In [350]:
from brickschema import Graph
from rdflib import Namespace
import os
from string import Template

# Validation Class

In [399]:

class BasicValidationInterface:

    def __init__(self, graph_path: str, nightly: bool = False):
        self.graph_path = graph_path
        if nightly:
            self.graph = Graph(load_brick_nightly=True)
        else:
            self.graph = Graph(load_brick=True)
        # loading graph in init
        self.graph.load_file(self.graph_path)
        # load qudt
        self.graph.parse('https://qudt.org/2.1/schema/qudt', format = 'ttl')
        self.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format = 'ttl')

    def validate(self):
        valid, report_graph, report = self.graph.validate()
        print(f"Graph <{self.graph_path}> is valid? {valid}")
        if not valid:
            print("-" * 79)
            print(report)
            print("-" * 79)
        return valid, report_graph


# Area correction query pattern

In [391]:
area_query_format  = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "$old" }
INSERT {?this brick:value $new }
WHERE {
?this brick:value "$old"
}
"""

res = report_graph.query(""" 
select ?replace
WHERE {
?s a sh:ValidationResult ;
    sh:value ?value .
?val brick:value ?replace .
}
""")
                         
def format_area_query(report_graph):
    res = report_graph.query(""" 
        select ?replace
        WHERE {
        ?s a sh:ValidationResult ;
            sh:value ?value .
        ?val brick:value ?replace .
        }
        """)
    if len(res) == 1:
        print('1 Invalid Shape')
        old = iter(res).__next__().get('replace')
        new = int(old.rsplit('^^')[0])
    else: 
        raise ValueError('multiple errors')
    return Template(area_query_format).substitute(old = old, new = new)


# Other Queries

In [ ]:
deprecation_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a ?class}
INSERT {?this a ?newClass}
WHERE {
?this a ?class .
?class owl:deprecated true .
?class brick:deprecatedInVersion ?depver .
?class brick:isReplacedBy ?newClass .
}
"""

In [ ]:
#
ft2_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:hasUnits unit:FT_2}
INSERT {?this brick:hasUnit unit:FT2}
WHERE {
?this brick:hasUnits unit:FT_2
}
"""

In [ ]:
chwv_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Valve_Command }
INSERT {?this a brick:Valve_Command ;
                brick:hasSubstance brick:Chilled_Water }
WHERE {
?this a brick:Chilled_Water_Valve_Command
}
"""

In [ ]:
hwv_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Hot_Water_Valve_Command }
INSERT {?this a brick:Valve_Command ;
                brick:hasSubstance brick:Hot_Water }
WHERE {
?this a brick:Hot_Water_Valve_Command
}
"""

In [ ]:
# ASSUMING that flow refers to a Sensor
bldg4_supflow_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Supply_Air_flow }
INSERT {?this a brick:Supply_Air_Flow_Sensor}
WHERE {
?this a brick:Supply_Air_flow
}
"""

In [ ]:
chws_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Supply_Temperature_Setpoint}
INSERT {?this a brick:Supply_Chilled_Water_Temperature_Setpoint }
WHERE {
?this a brick:Chilled_Water_Supply_Temperature_Setpoint
}
"""

In [ ]:
chwr_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Return_Temperature_Setpoint}
INSERT {?this a brick:Return_Chilled_Water_Temperature_Setpoint }
WHERE {
?this a brick:Chilled_Water_Return_Temperature_Setpoint
}
"""

In [ ]:
meters_fix ="""
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?meter brick:isPointOf ?this}
INSERT {?meter brick:meters ?this }
WHERE {
?meter a brick:Electric_Meter ;
    brick:isPointOf ?this .
}
"""

In [ ]:
electricalmeter_fix ="""
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?meter a brick:Electric_Meter }
INSERT {?meter a brick:Electrical_Meter  }
WHERE {
?meter a brick:Electric_Meter 
}
"""

# Correcting All the Buildings

In [392]:
# Deprecation query should come last
other_query_list = [chwv_fix, ft2_fix, bldg4_supflow_fix, hwv_fix, chws_fix, chwr_fix, meters_fix, electricalmeter_fix, deprecation_fix]

In [402]:
for file in os.listdir('graphs'):
    print(file)
    path = os.path.join('graphs',file)
    bv = BasicValidationInterface(graph_path=path, nightly=True)
    for q in other_query_list:
        bv.graph.update(q)
    valid, report_graph = bv.validate()
    if valid:
        print("Graph is valid")
        continue
    area_query = format_area_query(report_graph)
    bv.graph.update(area_query)
    valid, final_report = bv.validate()
    if not valid:
        break
    bv.graph.serialize(os.path.join('graphs_clean', file), format = 'ttl')

bldg5.ttl
Graph <graphs/bldg5.ttl> is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (1):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: bsh:hasareaShape
	Focus Node: ns2:bldg5
	Value Node: [ brick:hasUnit unit:FT2 ; brick:value Literal("99922^^<http://www.w3.org/2001/XMLSchema#integer>") ]
	Result Path: brick:area
	Message: Value does not conform to Shape bsh:AreaShape

-------------------------------------------------------------------------------
1 Invalid Shape
Graph <graphs/bldg5.ttl> is valid? True
bldg4.ttl
Graph <graphs/bldg4.ttl> is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (1):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Sou

ValueError: multiple errors

# Working it out

In [286]:
deprecation_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a ?class}
INSERT {?this a ?newClass}
WHERE {
?this a ?class .
?class owl:deprecated true .
?class brick:deprecatedInVersion ?depver .
?class brick:isReplacedBy ?newClass .
}
"""

In [287]:
#
ft2_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:hasUnits unit:FT_2}
INSERT {?this brick:hasUnit unit:FT2}
WHERE {
?this brick:hasUnits unit:FT_2
}
"""

In [288]:
bldg1_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 9973 }
WHERE {
?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>"
}
"""

In [289]:
#bldg 1

In [370]:
graph = 'graphs/bldg1.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(deprecation_fix)
# bv.graph.update(bldg1_area_fix)
bv.graph.update(ft2_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format = 'ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format = 'ttl')
report_graph = bv.validate()

Graph <graphs/bldg1.ttl> is valid? False
-------------------------------------------------------------------------------
Validation Report
Conforms: False
Results (1):
Constraint Violation in NodeConstraintComponent (http://www.w3.org/ns/shacl#NodeConstraintComponent):
	Severity: sh:Violation
	Source Shape: bsh:hasareaShape
	Focus Node: ns2:bldg1
	Value Node: [ brick:hasUnit unit:FT2 ; brick:value Literal("9973^^<http://www.w3.org/2001/XMLSchema#integer>") ]
	Result Path: brick:area
	Message: Value does not conform to Shape bsh:AreaShape

-------------------------------------------------------------------------------


In [388]:
new = format_area_query(report_graph)
print(new)

1 Invalid Shape

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 9973 }
WHERE {
?this brick:value "9973^^<http://www.w3.org/2001/XMLSchema#integer>"
}



In [389]:
bv.graph.update(new)

In [390]:
bv.validate()

Graph <graphs/bldg1.ttl> is valid? True


<Graph identifier=N8cc758c050614bad8073c97b06b3a6bc (<class 'rdflib.graph.Graph'>)>

In [348]:
if len(res) == 1:
    print('1 Invalid Shape')
    old = iter(res).__next__().get('replace')
    new = int(old.rsplit('^^')[0])

1 Invalid Shape


In [ ]:
Template(area_query_format).substitute(old = 'a', new = 'b')

In [298]:
bv.graph.serialize('graphs_clean/bldg1.ttl')

<Graph identifier=N596ed28253704a03bb8e96a39a54213f (<class 'brickschema.graph.Graph'>)>

In [140]:
# bldg2 

In [141]:
chwv_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Valve_Command }
INSERT {?this a brick:Valve_Command ;
                brick:hasSubstance brick:Chilled_Water }
WHERE {
?this a brick:Chilled_Water_Valve_Command
}
"""

In [142]:
hwv_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Hot_Water_Valve_Command }
INSERT {?this a brick:Valve_Command ;
                brick:hasSubstance brick:Hot_Water }
WHERE {
?this a brick:Hot_Water_Valve_Command
}
"""

In [143]:
chws_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Supply_Temperature_Setpoint}
INSERT {?this a brick:Supply_Chilled_Water_Temperature_Setpoint }
WHERE {
?this a brick:Chilled_Water_Supply_Temperature_Setpoint
}
"""

In [144]:
chwr_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Chilled_Water_Return_Temperature_Setpoint}
INSERT {?this a brick:Return_Chilled_Water_Temperature_Setpoint }
WHERE {
?this a brick:Chilled_Water_Return_Temperature_Setpoint
}
"""

In [145]:
meters_fix ="""
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?meter brick:isPointOf ?this}
INSERT {?meter brick:meters ?this }
WHERE {
?meter a brick:Electric_Meter ;
    brick:isPointOf ?this .
}
"""

In [146]:
electricalmeter_fix ="""
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?meter a brick:Electric_Meter }
INSERT {?meter a brick:Electrical_Meter  }
WHERE {
?meter a brick:Electric_Meter 
}
"""

In [233]:
# couldn't figure out how to generalize this (it is probably possible, but I couldn't crack it)
bldg2_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "26793^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 26793 }
WHERE {
?this brick:value "26793^^<http://www.w3.org/2001/XMLSchema#integer>"
}
"""

In [234]:
graph = 'graphs/bldg2.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(bldg2_area_fix)
bv.graph.update(chwv_fix)
bv.graph.update(ft2_fix)
bv.graph.update(bldg2_area_fix)
bv.graph.update(hwv_fix)
# bv.graph.update(bldg2_area_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format = 'ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format = 'ttl')

bv.graph.update(chws_fix)
bv.graph.update(chwr_fix)
bv.graph.update(meters_fix)
bv.graph.update(electricalmeter_fix)
bv.graph.update(deprecation_fix)
bv.validate()

Graph <graphs/bldg2.ttl> is valid? True


In [ ]:
bv.graph.serialize('graphs_clean/bldg2.ttl')

In [230]:
#bldg3

In [238]:
# couldn't figure out how to generalize this (it is probably possible, but I couldn't crack it)
bldg3_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "42946" }
INSERT {?this brick:value 42946 }
WHERE {
?this brick:value "42946"
}
"""

In [239]:
graph = 'graphs/bldg3.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(bldg1_area_fix)
bv.graph.update(chwv_fix)
bv.graph.update(ft2_fix)
bv.graph.update(bldg3_area_fix)
bv.graph.update(hwv_fix)
bv.graph.update(bldg2_area_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format = 'ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format = 'ttl')

bv.graph.update(chws_fix)
bv.graph.update(chwr_fix)
bv.graph.update(meters_fix)
bv.graph.update(electricalmeter_fix)
bv.graph.update(deprecation_fix)
bv.validate()

Graph <graphs/bldg3.ttl> is valid? True


In [ ]:
bv.graph.serialize('graphs_clean/bldg3.ttl')

In [240]:
#bldg4

In [251]:
# couldn't figure out how to generalize this (it is probably possible, but I couldn't crack it)
bldg4_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "32792^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 32792 }
WHERE {
?this brick:value "32792^^<http://www.w3.org/2001/XMLSchema#integer>"
}
"""

In [252]:
# ASSUMING that flow refers to a Sensor
bldg4_supflow_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a brick:Supply_Air_flow }
INSERT {?this a brick:Supply_Air_Flow_Sensor}
WHERE {
?this a brick:Supply_Air_flow
}
"""

In [253]:
graph = 'graphs/bldg4.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(bldg1_area_fix)
bv.graph.update(chwv_fix)
bv.graph.update(ft2_fix)
bv.graph.update(bldg4_supflow_fix)
bv.graph.update(bldg4_area_fix)
bv.graph.update(hwv_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format = 'ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format = 'ttl')

bv.graph.update(chws_fix)
bv.graph.update(chwr_fix)
bv.graph.update(meters_fix)
bv.graph.update(electricalmeter_fix)
bv.graph.update(deprecation_fix)
bv.validate()

Graph <graphs/bldg4.ttl> is valid? True


In [ ]:
bv.graph.serialize('graphs_clean/bldg4.ttl')

In [254]:
#bldg5

In [257]:
# couldn't figure out how to generalize this (it is probably possible, but I couldn't crack it)
bldg5_area_fix = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this brick:value "99922^^<http://www.w3.org/2001/XMLSchema#integer>" }
INSERT {?this brick:value 99922}
WHERE {
?this brick:value "99922^^<http://www.w3.org/2001/XMLSchema#integer>"
}
"""

In [258]:
graph = 'graphs/bldg5.ttl'
bv = BasicValidationInterface(graph_path=graph, nightly=True)
bv.graph.update(bldg1_area_fix)
bv.graph.update(chwv_fix)
bv.graph.update(ft2_fix)
bv.graph.update(bldg5_area_fix)
bv.graph.update(hwv_fix)
# Loading qudt fixes QUDT related validation errors
bv.graph.parse('https://qudt.org/2.1/schema/qudt', format = 'ttl')
bv.graph.parse('https://qudt.org/2.1/vocab/quantitykind', format = 'ttl')

bv.graph.update(chws_fix)
bv.graph.update(chwr_fix)
bv.graph.update(meters_fix)
bv.graph.update(electricalmeter_fix)
bv.graph.update(deprecation_fix)
bv.validate()

Graph <graphs/bldg5.ttl> is valid? True


In [260]:
bv.graph.serialize('graphs_clean/bldg5.ttl')

<Graph identifier=Nb9469e785c5e45fe8374b40bdde783d2 (<class 'brickschema.graph.Graph'>)>

In [ ]:
# Running all for rest of process

In [262]:
area_query_format = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this $old  }
INSERT {?this brick:value $new }
WHERE {
?this brick:value %old 
}
"""

In [265]:
Template(area_query_format).substitute(old = 'a', new = 'b')


PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extension/g36#>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bldg: <http://buildsys.org/ontologies/bldg1>

DELETE {?this a  }
INSERT {?this brick:value b }
WHERE {
?this brick:value %old 
}

